# Assignment Problem

A fundamental combinatorial optimization problem.

- $n$ tasks to be done
- $n$ workers to do the tasks
- Each worker has a certain cost for each task ($c_{ij}$)

Problem: Find the best assignment of all tasks that minimizes the total cost.

Constraints:
* Each worker can do at most one task
* All tasks need to be done

# Mathematical Formulation

\begin{align}
\text{minimize} & \sum_{i = 1}^{n}\sum_{j = 1}^{n} c_{ij}x_{ij}, \\
\text{subject to} & \\
& \sum_{i = 1}^{n} x_{ij} \leq 1 & \forall j = 1, ..., n & \ \ \textit{ (workload)},\\
& \sum_{j = 1}^{n} x_{ij} = 1 & \forall i = 1, ..., n & \ \ \textit{ (task completion)}, \\
& x_{ij} \in \{0,1\} & \forall i,j = 1, ..., n
\end{align}

where $x_{ij}$ is 1 if worker $i$ performs task $j$, 0 otherwise.

We can replace the integrality constraints with continuous constraints. The solution will not change as the constraint matrix is totally unimodular.

# Coding in Python using gurobipy
## Step 1: Creating input data (cost matrix)

In [2]:
import numpy as np
cost = np.random.randint(1, 10, (4,4))
print(cost.shape[0])
print(cost)

4
[[4 6 5 1]
 [7 4 2 4]
 [7 7 3 2]
 [1 3 7 4]]


## Step 2: Importing gurobipy package

In [2]:
from gurobipy import Model, GRB

## Step 3: Creating the model

In [3]:
assignment_model = Model('Assignment')

Set parameter Username
Set parameter LicenseID to value 2652794
Academic license - for non-commercial use only - expires 2026-04-15


## Step 4: Creating decision variables

In [4]:
x = assignment_model.addVars(cost.shape[0], 
                            cost.shape[1], 
                            vtype = GRB.BINARY, 
                            name = "x")

## Step 5: Adding the constraints
\begin{align}
\sum_{i = 1}^{n} x_{ij} \leq 1 &\qquad \forall j = 1, ..., n & \ \ \textit{ (workload)},\\
\sum_{j = 1}^{n} x_{ij} = 1 & \qquad \forall i = 1, ..., n & \ \ \textit{ (task completion)}.
\end{align}

In [5]:
# sum_{i = 1}^{n} x_{ij} <= 1 for all j
assignment_model.addConstrs((sum(x[i,j] for i in range(cost.shape[0])) <= 1 
                             for j in range(cost.shape[1])), 
                            name = 'work_load')

# sum_{j = 1}^{n} x_{ij}  = 1 for all i
assignment_model.addConstrs((sum(x[i,j] for j in range(cost.shape[1])) == 1 
                             for i in range(cost.shape[0])),
                            name = 'task_completion')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

## Step 6: Defining the objective function
$$\sum_{i = 1}^{n}\sum_{j = 1}^{n} c_{ij}x_{ij}$$

In [6]:
obj_fn = sum(cost[i,j]*x[i,j] for i in range(cost.shape[0]) for j in range(cost.shape[1]))

assignment_model.setObjective(obj_fn, GRB.MINIMIZE)

## Step 7: Solve and inspect the model

In [7]:
assignment_model.setParam('OutputFlag', False)
assignment_model.optimize()

print('Model Statistics')
assignment_model.printStats()
print('\n\nModel Output\n')
print(assignment_model.display())

Model Statistics


Model Output

None


C:\Users\phamp\AppData\Local\Temp\ipykernel_24580\486250653.py:7: DeprecationWarning: Model.display() is deprecated
  print(assignment_model.display())


## Output the solution

In [8]:
print('Optimization is done. Objective Function Value: %.2f' % assignment_model.objVal)

# Get values of the decision variables
for v in assignment_model.getVars():
    if v.x > 0:
        print('%s: %g' % (v.varName, v.x))

Optimization is done. Objective Function Value: 15.00
x[0,1]: 1
x[1,0]: 1
x[2,2]: 1
x[3,3]: 1


# Relaxing the binary constraint

In [9]:
assignment_model_2 = Model('Assignment_2')

y = assignment_model_2.addVars(cost.shape[0], 
                               cost.shape[1],
                               vtype = GRB.CONTINUOUS,
                               lb = 0, ub = 1, 
                               name = "x")

# sum_{i = 1}^{n} y_{ij} <= 1 for all j
assignment_model_2.addConstrs((sum(y[i,j] for i in range(cost.shape[0])) == 1 
                               for j in range(cost.shape[1])), 
                              name = 'work_load')

# sum_{j = 1}^{n} y_{ij}  = 1 for all i
assignment_model_2.addConstrs((sum(y[i,j] for j in range(cost.shape[1])) == 1 
                               for i in range(cost.shape[0])),
                              name = 'task_completion')

obj_fn = sum(cost[i,j]*y[i,j] for i in range(cost.shape[0]) for j in range(cost.shape[1]))
assignment_model_2.setObjective(obj_fn, GRB.MINIMIZE)

assignment_model_2.setParam('OutputFlag', False)

assignment_model_2.optimize()

print('Optimization is done. Objective Function Value: %.2f' % assignment_model_2.objVal)

# Get values of the decision variables
for v in assignment_model.getVars():
    if v.x > 0:
        print('%s: %g' % (v.varName, v.x))

Optimization is done. Objective Function Value: 15.00
x[0,1]: 1
x[1,0]: 1
x[2,2]: 1
x[3,3]: 1
